In [ ]:
import sys
sys.path.append('/home/jovyan/EngiOptiQA')

In [ ]:
import os

from datetime import datetime
from engioptiqa import AnnealingSolverDWave, Rod1D, StructuralAnalysisProblem
from matplotlib import pyplot as plt
import numpy as np


In [ ]:
# Create an output folder with current time stamp.
notebook_directory = os.path.abspath("")
timestamp = datetime.now().strftime("%Y_%m_%d_%H-%M-%S")
output_path = f"{timestamp}_output"
output_path = os.path.join(notebook_directory, output_path)
os.makedirs(output_path)
output_path_dwave = os.path.join(output_path,'dwave')

# Solving a Structural Analysis Problem with EngiOptiQA

In this notebook, you can solve the structural analysis problem presented in [Key and Freinberger (2024)](https://doi.org/10.3390/math12030482) using *Quantum Annealing (QA)*.

## Suggested Steps
   1. Reproduce the results from the paper
   2. Compare with the results from Simulated Annealing (SA)
   3. Study the effect of changing 
      - the number of qubits for representig the real-valued variables (`n_qubits_per_node`)
      - the penalty weight for the constraint of static admissibility (`penalty_weight_dwave`)
      - the number of reads (`num_reads`)
      - the annealing time (`annealing_time`)
      - the chain strength (`chain_strength`)
      - ...


## The Structural Analysis Problem

In [ ]:
# Define the structural analysis problem for one-dimensional rod under self-weight loading through body force density g.
g = 2.5
# Rod with n_comp components and of length L.
n_comp = 5; L = 1.5; A = 0.25; rod_1d = Rod1D(n_comp, L, A)

analysis_problem_dwave = StructuralAnalysisProblem(rod_1d, g, output_path=output_path_dwave)

### Analytical Solution

In [ ]:
# Compute analytical solution.
analysis_problem_dwave.compute_analytical_solution()

### Numerical Solution

#### DWave

In [ ]:
# Setup the annealing solver.
token_files = {
    'DWave': './token_DWave.txt'
    }

annealing_solver_dwave = AnnealingSolverDWave(token_files['DWave'])
annealing_solver_dwave.setup_solver(solver_type='qpu')

#### Discretization through Real-Valued Nodal Coefficients

In [ ]:
# Discretization.
binary_representation = 'normalized'
n_qubits_per_node = 10

analysis_problem_dwave.generate_discretization(n_qubits_per_node, binary_representation)

#### QUBO Formulation

In [ ]:
# QUBO formulation.
penalty_weight_dwave = 2e1 # effective penalty weight
analysis_problem_dwave.generate_qubo_formulation(penalty_weight=penalty_weight_dwave)
analysis_problem_dwave.visualize_qubo_matrix_pattern(highlight_interactions=True, save_fig=False, save_tikz=False)

#### Transform the Problem from the Amplify SDK to the DWave SDK

In [ ]:
# Transform Amplify problem for DWave
lp_file_name = 'structural_analysis_problem.lp'
lp_file_path = os.path.join(output_path, lp_file_name)
analysis_problem_dwave.transform_to_dwave(lp_file_path)


#### Perform the Annealing

In [ ]:
# Solve problem.
annealing_solver_dwave.solve_qubo_problem(
    analysis_problem_dwave,
    num_reads=500,
    annealing_time = 400.0,
    auto_scale = True,
    label = f"Structural Analysis Problem",
    return_embedding = False,
    chain_strength = 50
    )


#### Analyze the Raw Results

In [ ]:
solutions_dwave = analysis_problem_dwave.analyze_results(result_max=0)

In [ ]:
errors_l2 = [d['error_l2_rel'] for d in solutions_dwave]
errors_h1 = [d['error_h1_rel'] for d in solutions_dwave]
objectives = [d['objective'] for d in solutions_dwave]
complementary_energy = [d['complementary_energy'] for d in solutions_dwave]
complementary_energy_diff_rel = [np.abs(d['complementary_energy']-analysis_problem_dwave.PI_analytic)/np.abs(analysis_problem_dwave.PI_analytic) for d in solutions_dwave]
constraints = [d['constraints'] for d in solutions_dwave]
constraints_weighted = [penalty_weight_dwave*d['constraints'] for d in solutions_dwave]

#### Update the Penalty Weight

In [ ]:
# Perform local search with adapted penalty weight.
analysis_problem_dwave.update_penalty_weight_in_qubo_formulation(penalty_weight=1.e9)
annealing_solver_dwave.perform_local_search(analysis_problem_dwave) 

#### Analyze the Post-Processed Results

In [ ]:
# Analyze post-processed results
solutions_dwave_pp = analysis_problem_dwave.analyze_results(results=analysis_problem_dwave.results_pp, result_max=0)


errors_l2_pp = [d['error_l2_rel'] for d in solutions_dwave_pp]
errors_h1_pp = [d['error_h1_rel'] for d in solutions_dwave_pp]
objectives_pp = [d['objective'] for d in solutions_dwave_pp]
complementary_energy_pp = [d['complementary_energy'] for d in solutions_dwave_pp]
complementary_energy_diff_rel_pp = [np.abs(d['complementary_energy']-analysis_problem_dwave.PI_analytic)/np.abs(analysis_problem_dwave.PI_analytic) for d in solutions_dwave_pp]
constraint_pp = [d['constraints'] for d in solutions_dwave_pp]

In [ ]:
i_min =np.argsort(objectives_pp)
i_sol = i_min[0]
solution = solutions_dwave_pp[i_sol]
error_l2 = errors_l2_pp[i_sol]
error_h1 = errors_h1_pp[i_sol]
objective = objectives_pp[i_sol]
comp_energy_diff_rel =  complementary_energy_diff_rel_pp[i_sol]

In [ ]:
analysis_problem_dwave.plot_force(
    analysis_problem_dwave.force_analytic, 
    solution['force'], 
    subtitle='Quantum Annealing',
    file_name=os.path.join(output_path_dwave, 'force_quantum_annealing'),
    save_fig = True,
    save_tikz = False
) 